In [1]:
import os 
from PIL import Image 
from arguments import parser 
import torch 
import torch.nn as nn 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.manifold import TSNE
from datasets import create_dataset 
from torch.utils.data import DataLoader
from utils.metrics import MetricCalculator, loco_auroc
from accelerate import Accelerator
from omegaconf import OmegaConf
import seaborn as sns 

os.environ['CUDA_VISIBLE_DEVICES'] = '0' 
default_setting = './configs/default/mvtecad.yaml'
model_setting = './configs/model/CAE.yaml'
cfg = parser(True,default_setting, model_setting)


model  = __import__('models').__dict__[cfg.MODEL.method](
        backbone = cfg.MODEL.backbone,
        **cfg.MODEL.params
        ).to('cuda')
device = 'cuda'


loader_dict = {}
accelerator = Accelerator()
for cn in cfg.DATASET.class_names:
    trainset, testset = create_dataset(
        dataset_name  = cfg.DATASET.dataset_name,
        datadir       = cfg.DATASET.datadir,
        class_name    = cn,
        img_size      = cfg.DATASET.img_size,
        mean          = cfg.DATASET.mean,
        std           = cfg.DATASET.std,
        aug_info      = cfg.DATASET.aug_info,
        **cfg.DATASET.get('params',{})
    )
    trainloader = DataLoader(
        dataset     = trainset,
        batch_size  = cfg.DATASET.batch_size,
        num_workers = cfg.DATASET.num_workers,
        shuffle     = True 
    )    

    testloader = DataLoader(
            dataset     = testset,
            batch_size  = 8,
            num_workers = cfg.DATASET.num_workers,
            shuffle     = False 
        )    
    
    loader_dict[cn] = {'train':trainloader,'test':testloader}    


 Experiment Name : .-Continual_True-online_False



/opt/conda/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
imgs, labels = next(iter(trainloader))
model = model.to('cuda')
imgs = imgs.to('cuda')
outputs = model(imgs)
loss = model.criterion(outputs)

In [8]:
len(outputs)

3

/Volume/VAD/LifeLongerAD_cu121/models/cae.py:112: UserWarning: Using a target size (torch.Size([32, 64, 28, 28])) that is different to the input size (torch.Size([32, 3, 224, 224])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(decoder_features, encoder_features)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 outputs = model(imgs)                                                                        │
│ ❱ 2 loss = model.criterion(outputs)                                                              │
│   3                                                                                              │
│                                                                                                  │
│ /Volume/VAD/LifeLongerAD_cu121/models/cae.py:119 in criterion                                    │
│                                                                                                  │
│   116 │                                                                                          │
│   117 │   def criterion(self, outputs: tuple):                                                   │
│   118 │   │   (encoder_features, decoder_features) = outputs                                     │
│ ❱ 119 │   │   loss = self._criterion(encoder_features, decoder_features)                         │
│   120 │   │   return loss                                                                        │
│   121 │                                                                                          │
│   122 │   def get_score_map(self, outputs: tuple):                                               │
│                                                                                                  │
│ /Volume/VAD/LifeLongerAD_cu121/models/cae.py:112 in _default_criterion                           │
│                                                                                                  │
│   109 │   │   │   return self.encoder(x)                                                         │
│   110 │                                                                                          │
│   111 │   def _default_criterion(self, encoder_features, decoder_features):                      │
│ ❱ 112 │   │   return F.mse_loss(decoder_features, encoder_features)                              │
│   113 │                                                                                          │
│   114 │   def _default_anomaly_map_generator(self, encoder_features, decoder_features):          │
│   115 │   │   return torch.abs(encoder_features - decoder_features).mean(dim=1, keepdim=True)    │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/torch/nn/functional.py:3791 in mse_loss                  │
│                                                                                                  │
│   3788 │   if size_average is not None or reduce is not None:                                    │
│   3789 │   │   reduction = _Reduction.legacy_get_string(size_average, reduce)                    │
│   3790 │                                                                                         │
│ ❱ 3791 │   expanded_input, expanded_target = torch.broadcast_tensors(input, target)              │
│   3792 │   return torch._C._nn.mse_loss(                                                         │
│   3793 │   │   expanded_input, expanded_target, _Reduction.get_enum(reduction)                   │
│   3794 │   )                                                                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/torch/functional.py:76 in broadcast_tensors              │
│                                                                                                  │
│     73 │   # This wrapper exists to support variadic args. 

In [7]:
outputs[0].shape

torch.Size([32, 64, 28, 28])